<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Topic_Modeling_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

In [1]:
!pip install -U kaleido
!pip install bertopic
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 79.9 MB 123 kB/s 
     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 88 kB 4.1 MB/s 
     |████████████████████████████████| 5.2 MB 42.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 4.0 MB 34.5 MB/s 
     |████████████████████████████████| 1.2 MB 40.2 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 40.3 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 
     |████████████████████████████████| 636 kB 47.2 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330764 sha256=bd12c052b683b9937e897421b621c4dad6f3c545ad7e25956ae5697be4367717
  Stored in director

In [2]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido

multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, doc):
    return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# # using spacy

# nlp = spacy.load('en_core_web_lg')

# class LemmaTokenizer:
#   def __init__(self):
#     self.wnl = WordNetLemmatizer()
#   def __call__(self, doc):
#     # return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

#     return [self.wnl.t.lemma_ for t in nlp(doc)]

## Define path of files

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

In [6]:
files[-5]

'Beamline_8.3.1.xls'

## Helper Functions

In [7]:
def create_input(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']
  combined = list(df['Combined'])

  # remove elements with blank
  while " " in combined:
    combined.remove(" ")

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, ' ', l )
    comb_clean.append(mod_string)


  return comb_clean

In [8]:
# def lemma_spacy(df_combined):

#   new_df= []
#   for doc in df_combined:
#     filt_combined = []
#     for word in nlp(doc):
#       filt_combined.append(word.lemma_)

#     new_df.append(" ".join(filt_combined))

#   return new_df

  

In [9]:
def remove_stop(df_combined):

  # remove stopwords

  new_df= []
  for doc in df_combined:
    filt_combined = []
    for word in word_tokenize(doc):

      if word.lower() not in stopwords.words('english'):
        # print(word)
        if word.lower() == "perovskites":
          filt_combined.append("perovskite")
        else:
          filt_combined.append(word)
    new_df.append(" ".join(filt_combined))

  return new_df

In [10]:
def berttopic_basic(df):

  topic_model = BERTopic()
  topics, probs = topic_model.fit_transform(df)

  # added lemmatization
  # update using lemmatization

  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model

In [11]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

## example

In [ ]:
df = pd.read_table(path + "Beamline_7.3.3.xls", on_bad_lines='skip')

# combine abstract and title
input_data = create_input(df)

# remove stopwords
filtered_input = remove_stop(input_data)

NameError: ignored

## Using BERTopic

In [ ]:
start_time = time.time()
iter_version = "BERTopic_v2/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # run topic modeling algorithm
  model = berttopic_basic(filtered_input)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))


File: 1 Beamline_1.4.xls


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


File: 1 done 
File: 2 Beamline_10.0.1.xls
File: 2 done 
File: 3 Beamline_10.3.1.xls
File: 3 done 
File: 4 Beamline_10.3.2.xls
File: 4 done 
File: 5 Beamline_11.0.1.xls
File: 5 done 
File: 6 Beamline_11.0.2.xls
File: 6 done 
File: 7 Beamline_11.3.1.xls
File: 7 done 
File: 8 Beamline_11.3.2.xls
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
File: 9 done 
File: 10 Beamline_12.0.1.2.xls
File: 10 done 
File: 11 Beamline_12.0.1.4.xls
File: 11 done 
File: 12 Beamline_12.0.2.xls
File: 12 done 
File: 13 Beamline_12.2.1.xls
File: 13 done 
File: 14 Beamline_12.2.2.xls
File: 14 done 
File: 15 Beamline_12.3.1.xls
File: 15 done 
File: 16 Beamline_12.3.2.xls
File: 16 done 
File: 17 Beamline_2.1.xls
File: 17 done 
File: 18 Beamline_3.1.1.xls
File: 18 done 
File: 19 Beamline_3.3.2.xls
File: 19 done 
File: 20 Beamline_4.0.2.xls
File: 20 done 
File: 21 Beamline_4.0.3.1.xls
File: 21 done 
File: 22 Beamline_4.0.3.2.xls
File: 22 done 
File: 23 Beamline_4.2.2.xls
File: 23 done 
File: 24 Beamline_5.0.1.xls
File:

## Using SPECTER

In [ ]:
def remove_punct(df):

  # remove punctuation
  text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  return text_col


In [ ]:
def berttopic_specter(df):

  sentence_model = SentenceTransformer('allenai-specter')
  topic_model_specter = BERTopic(embedding_model=sentence_model)

  topics, probs = topic_model_specter.fit_transform(df)

  # added lemmatization
  # update using lemmatization


  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  if len(topic_model_specter.topics)==1:
    if -1 in list(topic_model_specter.topics.keys()):
      # retrain model 
      topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
      topics, probs = topic_model_specter.fit_transform(filtered_input_2)

      tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
      # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
      topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model_specter

In [ ]:
# df = pd.read_table(path + "Beamline_4.0.3.1.xls", on_bad_lines='skip')
# # df = pd.read_table(path + "Beamline_11.0.1.xls", on_bad_lines='skip')


# # # combine abstract and title
# # input_data = create_input(df)

# # # remove stopwords
# # filtered_input = remove_stop(input_data)

# # filtered_input_2 = remove_punct(filtered_input)

# sentence_model = SentenceTransformer('allenai-specter')
# topic_model_specter = BERTopic(embedding_model=sentence_model)

# topics, probs = topic_model_specter.fit_transform(filtered_input_2)

# print(topic_model_specter.topics)



# if len(topic_model_specter.topics)==1:
#   if -1 in list(topic_model_specter.topics.keys()):
#     # retrain model 
#     topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
#     topics, probs = topic_model_specter.fit_transform(filtered_input_2)

#     tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
#     # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
#     topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)



# # run topic modeling algorithm
# # model = berttopic_specter(filtered_input_2)



{-1: [('ultrafast', 1.6801020957902655), ('small', 0.9526677587279724), ('repetition', 0.2620700961367291), ('khz', 0.2620700961367291), ('computers', 0.2620700961367291), ('mhz', 0.2501466367149557), ('efficient', 0.2370786223680821), ('energy', 0.23352710838437443), ('center', 0.22976590893320176), ('faster', 0.22523404980752915)], 0: [('solar', 0.03933516389276976), ('cells', 0.0379834677124056), ('polymer', 0.035142449804982635), ('organic', 0.028611481450864132), ('morphology', 0.026961603414980453), ('performance', 0.025014747584466004), ('efficiency', 0.02358458779618354), ('high', 0.022362935801789857), ('acceptor', 0.02228680688855267), ('molecular', 0.021089679284766703)], 1: [('magnetic', 0.10446360928999067), ('spin', 0.052660301488942074), ('artificial', 0.03177173293315308), ('state', 0.03109486516617621), ('temperature', 0.029942083407576685), ('domain', 0.029206302336515878), ('antiferromagnetic', 0.027065696542852014), ('fe', 0.02561574851377066), ('thin', 0.0247265035

In [ ]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_v1/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # remove punctutation
  filtered_input_2 = remove_punct(filtered_input)

  # run topic modeling algorithm
  model = berttopic_specter(filtered_input_2)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))


File: 1 Beamline_1.4.xls
File: 1 done 
File: 2 Beamline_10.0.1.xls
File: 2 done 
File: 3 Beamline_10.3.1.xls
File: 3 done 
File: 4 Beamline_10.3.2.xls
File: 4 done 
File: 5 Beamline_11.0.1.xls
File: 5 done 
File: 6 Beamline_11.0.2.xls
File: 6 done 
File: 7 Beamline_11.3.1.xls
File: 7 done 
File: 8 Beamline_11.3.2.xls
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
File: 9 done 
File: 10 Beamline_12.0.1.2.xls
File: 10 done 
File: 11 Beamline_12.0.1.4.xls
File: 11 done 
File: 12 Beamline_12.0.2.xls
File: 12 done 
File: 13 Beamline_12.2.1.xls
File: 13 done 
File: 14 Beamline_12.2.2.xls
File: 14 done 
File: 15 Beamline_12.3.1.xls
File: 15 done 
File: 16 Beamline_12.3.2.xls
File: 16 done 
File: 17 Beamline_2.1.xls
File: 17 done 
File: 18 Beamline_3.1.1.xls
File: 18 done 
File: 19 Beamline_3.3.2.xls
File: 19 done 
File: 20 Beamline_4.0.2.xls
File: 20 done 
File: 21 Beamline_4.0.3.1.xls
File: 21 done 
File: 22 Beamline_4.0.3.2.xls
File: 22 done 
File: 23 Beamline_4.2.2.xls
File: 23 done 
File: 24

## SCI BERT

In [ ]:
def berttopic_scibert(df):

  scibert_emb = TransformerDocumentEmbeddings('allenai/scibert_scivocab_uncased')
  topic_model_specter = BERTopic(embedding_model=sentence_model)

  topics, probs = topic_model_specter.fit_transform(df)

  # added lemmatization
  # update using lemmatization


  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  if len(topic_model_specter.topics)==1:
    if -1 in list(topic_model_specter.topics.keys()):
      # retrain model 
      topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
      topics, probs = topic_model_specter.fit_transform(filtered_input_2)

      tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
      # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
      topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model_specter

In [ ]:
scibert_emb = TransformerDocumentEmbeddings('allenai/scibert_scivocab_uncased')


## getting most representative docs

In [ ]:
!pip install flair!pip install flair

     |████████████████████████████████| 401 kB 7.4 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 1.6 MB 44.6 MB/s 
     |████████████████████████████████| 1.2 MB 47.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 788 kB 51.9 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 981 kB 39.0 MB/s 
     |████████████████████████████████| 48 kB 4.7 MB/s 
     |████████████████████████████████| 199 kB 45.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=5840b98f9c4f8e325e2c1d83f5087d4a12eb14c7c2ba5364837035bb89f52a7d
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8

In [145]:
# df = pd.read_table(path + "Beamline_4.0.3.1.xls", on_bad_lines='skip')
df = pd.read_table(path + "Beamline_8.3.2.xls", on_bad_lines='skip')


# combine abstract and title
input_data = create_input(df)

# remove stopwords
filtered_input = remove_stop(input_data)

filtered_input_2 = remove_punct(filtered_input)

sentence_model = SentenceTransformer('allenai-specter')
# from flair.embeddings import TransformerDocumentEmbeddings

# scibert_emb = TransformerDocumentEmbeddings('allenai/scibert_scivocab_uncased')

# topic_model_specter = BERTopic(embedding_model=scibert_emb)
topic_model_specter = BERTopic(embedding_model=sentence_model)

topics, probs = topic_model_specter.fit_transform(filtered_input_2)

tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
# vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
topic_model_specter.update_topics(filtered_input_2, topics, vectorizer_model=tf_vectorizer)

# run topic modeling algorithm
# model = berttopic_specter(filtered_input_2)



2022-04-15 20:44:31,992 - BERTopic - Transformed documents to Embeddings
2022-04-15 20:44:36,115 - BERTopic - Reduced dimensionality with UMAP
2022-04-15 20:44:36,140 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [146]:
topic_model_specter.topics

{-1: [('sample', 0.024775084629347785),
  ('column', 0.02419944464058891),
  ('fe', 0.023794245996407816),
  ('shale', 0.022949063065883492),
  ('xray', 0.022785275183726215),
  ('grain', 0.02140996524404361),
  ('using', 0.020954644677502944),
  ('water', 0.020599601295563336),
  ('pumice', 0.019685760053140294),
  ('phase', 0.019620270282870667)],
 0: [('xray', 0.02999935900977287),
  ('material', 0.02853961887288175),
  ('data', 0.022767375045333983),
  ('composite', 0.022380392862905586),
  ('fiber', 0.020957947266650956),
  ('scaffold', 0.02050712321275014),
  ('tomography', 0.020266031184448922),
  ('mechanical', 0.019598324456635834),
  ('image', 0.019059026036234057),
  ('using', 0.01841851318362736)],
 1: [('lithium', 0.044873489812194245),
  ('electrolyte', 0.04420672282015106),
  ('cell', 0.03734441584535646),
  ('electrode', 0.034504692879023656),
  ('fuel', 0.03439586097017946),
  ('battery', 0.03285890266357192),
  ('layer', 0.02819481534413599),
  ('density', 0.026891164

In [147]:
topic_model_specter.get_representative_docs(1)

['Impact Salt Concentration Nonuniform Lithium Electrodeposition Rigid Block Copolymer Electrolytes growing demand higher energy density lithium batteries  One approach addressing demand enabling lithium metal anodes  However  nucleation growth electronically conductive protrusions  cause short circuits  prevent use technology liquid electrolytes  use rigid solid electrolytes polystyrenebpoly  ethylene oxide  electrolytes one solution  additional requirement practical cells needed use electrolytes high salt concentration maximize flux lithium ions cell  first systematic study effect salt concentration morphology electrodeposited lithium rigid block copolymer electrolyte presented  nature  areal density  morphologies defective lithium deposits created galvanostatic cycling lithiumlithium symmetric cells determined using hard Xray microtomography  Cycle life decreases rapidly increasing salt concentration  Xray microtomography reveals presence multiglobular protrusions  nucleated impurit

In [154]:
similar_topics, similarity = topic_model_specter.find_topics("ion", top_n=5)

In [155]:
similar_topics

[4, 1, -1, 5, 8]

In [156]:
topic_model_specter.get_topic(similar_topics[0])

[('carbonate', 0.041454505648516646),
 ('dissolution', 0.03924897140511595),
 ('permeability', 0.03843836098754812),
 ('experiment', 0.03834439726525035),
 ('fracture', 0.034823013615219674),
 ('reactive', 0.03437690712316015),
 ('rate', 0.03327423089447116),
 ('sediment', 0.03293918065027628),
 ('pore', 0.032933761627535175),
 ('cement', 0.03235473611591543)]

## Topic Modeling over time 

In [12]:
def create_input_new(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']

  # remove blanks
  df = df[df['Combined']!=" "]

  #keep only selected cols
  df_sel = df[['Authors','Pub Year','Research Area','Combined']]
  df_sel= df_sel.rename(columns={'Pub Year':'pub_year',"Research Area":"research_area","Authors":"authors","Combined":"combined"})

  combined = list(df_sel['combined'])

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, ' ', l )
    comb_clean.append(mod_string)

  # merge back to df
  df_sel['combined'] = comb_clean

  # convert years to int
  df_sel['pub_year'] = df_sel['pub_year'].astype(int)

  return df_sel

In [13]:
def remove_stop_df(df_combined):

  # remove stopwords

  new_df= []
  filt_combined = []
  for word in word_tokenize(df_combined):

    if word.lower() not in stopwords.words('english'):
      # print(word)
      if word.lower() == "perovskites":
        filt_combined.append("perovskite")
      else:
        filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

  

In [20]:
start_time = time.time()
iter_version = "BERTopic_v3/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # input_data['combined'] = input_data['combined'].apply(remove_punct)

  topic_model = BERTopic(verbose=True)
  list_text = input_data['combined'].to_list()
  topics, probs = topic_model.fit_transform(list_text)

  years = input_data['pub_year'].to_list()

  # defining path
  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(topic_model, file_path) 

  # dynamic topic modeling
  topics_over_time = topic_model.topics_over_time(list_text, topics, years)
  fig_time = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)
  fig_name_png = model_path + "topic_time" + beam_name  +".png"
  fig_time.write_image(fig_name_png)
  fig_name_html = model_path + "topic_time" + beam_name  +".html"
  fig_time.write_html(fig_name_html)

  # visualize barchart of topics as html
  fig = topic_model.visualize_barchart(top_n_topics = len(topic_model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(topic_model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))



File: 1 Beamline_1.4.xls


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2022-04-16 00:38:08,495 - BERTopic - Transformed documents to Embeddings
2022-04-16 00:38:12,917 - BERTopic - Reduced dimensionality with UMAP
2022-04-16 00:38:12,945 - BERTopic - Clustered UMAP embeddings with HDBSCAN
27it [00:00, 41.24it/s]


File: 1 done 
File: 2 Beamline_10.0.1.xls


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

...

File: 8 done 
File: 9 Beamline_12.0.1.1.xls


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2022-04-16 00:41:44,085 - BERTopic - Transformed documents to Embeddings
2022-04-16 00:41:47,864 - BERTopic - Reduced dimensionality with UMAP
2022-04-16 00:41:47,888 - BERTopic - Clustered UMAP embeddings with HDBSCAN
27it [00:00, 62.51it/s]


File: 9 done 
File: 10 Beamline_12.0.1.2.xls


ValueError: ignored

In [126]:
df = pd.read_table(path + "Beamline_8.3.2.xls", on_bad_lines='skip')
# df = pd.read_table(path + "Beamline_11.0.1.xls", on_bad_lines='skip')


# combine abstract and title
input_data = create_input_new(df)

# remove stopwords
input_data['combined'] = input_data['combined'].apply(remove_stop)

# input_data['combined'] = input_data['combined'].apply(remove_punct)



topic_model = BERTopic(verbose=True)
list_text = input_data['combined'].to_list()
topics, probs = topic_model.fit_transform(list_text)

years = input_data['pub_year'].to_list()



topics_over_time = topic_model.topics_over_time(list_text, topics, years)

# if len(topic_model_specter.topics)==1:
#   if -1 in list(topic_model_specter.topics.keys()):
#     # retrain model 
#     topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
#     topics, probs = topic_model_specter.fit_transform(filtered_input_2)

#     tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
#     # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
#     topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)



# run topic modeling algorithm
# model = berttopic_specter(filtered_input_2)



Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-04-15 19:57:24,878 - BERTopic - Transformed documents to Embeddings
2022-04-15 19:57:31,966 - BERTopic - Reduced dimensionality with UMAP
2022-04-15 19:57:31,995 - BERTopic - Clustered UMAP embeddings with HDBSCAN
19it [00:00, 33.33it/s]


In [127]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [131]:
topics_over_time_new = topic_model.topics_over_time(list_text, topics, years,global_tuning=True, evolution_tuning=False)
topic_model.visualize_topics_over_time(topics_over_time_new, top_n_topics=20)


19it [00:00, 29.86it/s]


In [119]:
def remove_stop(df_combined):

  # remove stopwords

  new_df= []
  filt_combined = []
  for word in word_tokenize(df_combined):

    if word.lower() not in stopwords.words('english'):
      # print(word)
      if word.lower() == "perovskites":
        filt_combined.append("perovskite")
      else:
        filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

In [ ]:
def berttopic_basic(df):

  topic_model = BERTopic()
  topics, probs = topic_model.fit_transform(df)

  # added lemmatization
  # update using lemmatization

  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model

In [111]:
input_data

,authors,pub_year,research_area,combined
0,"Larson, Natalie M.,Zok, Frank ,",2018,Materials - Condensed matter,Insights from in-situ x-ray computed tomograph...
1,"Nalla, Ravi K.,Kruzic, J.J. ,Kinney, John H.,R...",2004,None,Effect of aging on the toughness of human cort...
2,"Kinney, John H.,Nalla, Ravi K.,Pople, J.A. ,Br...",2005,None,Age related transparent Root Dentin: Mineral C...
4,"Balooch, Guive ,Balooch, Mehdi ,Nalla, Ravi K....",2005,Bioscience,TGF-beta regulates the mechanical properties a...
5,"Kinney, John H.,Stolken, J.S. ,Smith, T.S. ,Ry...",2005,Bioscience,An Orientation Distribution function for trabe...
...,...,...,...,...
506,"Yakovlev, M.A. ,Vanselow, D.J. ,Ngu, M.S. ,Zai...",2022,Applied Science - techniques,A wide-field micro-computed tomography detecto...
507,"Boigne, E. ,Bennett, N. ,Wang, A. ,Ihme, M. ,",2021,Energy Science,Examining the effect of fire retardant on the ...
508,"Li, Y. ,Parkinson, D.Y. ,Feng, J. ,Xia, C.-h. ...",2021,Bioscience,Quantitative X-ray tomographic analysis reveal...
509,"Fanton Borges, A.C. ,",2021,Bioscience,Mechanisms of Pierce's Disease Resistance in G...


In [122]:
filtered_input

0      Insights in-situ x-ray computed tomography axi...
1      Effect aging toughness human cortical bone : E...
2      Age related transparent Root Dentin : Mineral ...
4      TGF-beta regulates mechanical properties compo...
5      Orientation Distribution function trabecular b...
                             ...                        
506    wide-field micro-computed tomography detector ...
507    Examining effect fire retardant combustion woo...
508    Quantitative X-ray tomographic analysis reveal...
509    Mechanisms Pierce 's Disease Resistance Grapev...
510    Cracking pyrolysis preceramic polymers within ...
Name: combined, Length: 510, dtype: object